In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Eczema"
cohort = "GSE32924"

# Input paths
in_trait_dir = "../../input/GEO/Eczema"
in_cohort_dir = "../../input/GEO/Eczema/GSE32924"

# Output paths
out_data_file = "../../output/preprocess/Eczema/GSE32924.csv"
out_gene_data_file = "../../output/preprocess/Eczema/gene_data/GSE32924.csv"
out_clinical_data_file = "../../output/preprocess/Eczema/clinical_data/GSE32924.csv"
json_path = "../../output/preprocess/Eczema/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Nonlesional atopic dermatitis skin is characterized by broad terminal differentiation defects and variable immune abnormalities"
!Series_summary	"Atopic dermatitis (AD) is a common inflammatory skin disease with a T(H)2 and T22 immune polarity. Despite recent data showing a genetic predisposition to epidermal barrier defects in some patients, a fundamental debate still exists regarding the role of barrier abnormalities versus immune responses in initiating the disease. An extensive study of nonlesional AD (ANL) skin is necessary to explore whether there is an intrinsic predisposition to barrier abnormalities, background immune activation, or both in patients with AD. We sought to characterize ANL skin by determining whether epidermal differentiation and immune abnormalities that characterize lesional AD (AL) skin are also reflected in ANL skin.  We performed genomic and histologic profiling of both ANL and AL skin lesions (n = 12 each) compared wi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this appears to be a genomic profiling study
# comparing lesional and non-lesional skin in atopic dermatitis patients
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait (Eczema) data appears to be in row 2 as 'condition'
trait_row = 2

# Age data is not available in the sample characteristics
age_row = None

# Gender data is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert trait values to binary (0 for non-eczema, 1 for eczema)
    """
    if value is None or pd.isna(value):
        return None
    
    # Handle if value is a float or other non-string type
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert condition to binary values
    if value == 'Normal':
        return 0  # No eczema (control)
    elif value in ['ANL', 'AL']:
        return 1  # Eczema (both lesional and non-lesional skin from patients)
    else:
        return None  # Unknown or undefined condition

def convert_age(value):
    """
    Not used since age data is not available
    """
    return None

def convert_gender(value):
    """
    Not used since gender data is not available
    """
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering on usability of the dataset
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Perform this step only if trait_row is not None
if trait_row is not None:
    # Create sample characteristics dictionary based on the output from previous step
    sample_characteristics = {
        0: ['tissue: skin'], 
        1: ['individual: Patient P10', 'individual: Patient P13', 'individual: Patient P1', 
            'individual: Patient P14', 'individual: Patient P5', 'individual: Patient P7', 
            'individual: Patient P9', 'individual: Patient P2', 'individual: Patient P8', 
            'individual: Patient P11', 'individual: Patient P6', 'individual: Patient P12', 
            'individual: Patient P3', 'individual: Patient P4', 'individual: Patient N-1', 
            'individual: Patient N-2', 'individual: Patient N-3', 'individual: Patient N-4', 
            'individual: Patient N-5', 'individual: Patient N-6', 'individual: Patient N-7', 
            'individual: Patient N-8'], 
        2: ['condition: ANL', 'condition: AL', 'condition: Normal'], 
        3: ['eczema severity: SCORAD: 0', 'eczema severity: SCORAD: 37', 'eczema severity: SCORAD: 51', 
            'eczema severity: SCORAD: 36', 'eczema severity: SCORAD: 33', 'eczema severity: SCORAD: 57', 
            'eczema severity: SCORAD: 56', 'eczema severity: SCORAD: 55', 'eczema severity: SCORAD: 28', 
            'eczema severity: SCORAD: 64', 'eczema severity: SCORAD: 70', 'eczema severity: SCORAD: 91.1', 
            'eczema severity: SCORAD: 81', 'eczema severity: SCORAD: 97.5']
    }
    
    # Reconstruct clinical data in a format suitable for geo_select_clinical_features
    # Based on sample characteristics format from previous steps
    # We need to create a dataframe where:
    # - Rows are characteristics (indexed by row numbers)
    # - Columns are samples
    
    # First, determine the maximum number of values across all rows
    max_values = max(len(values) for values in sample_characteristics.values())
    
    # Create sample IDs
    sample_ids = [f"GSM{i}" for i in range(1, max_values+1)]
    
    # Create the DataFrame with proper structure
    clinical_data = pd.DataFrame(index=range(len(sample_characteristics)), columns=sample_ids)
    
    # Fill the DataFrame with values
    for row_num, values in sample_characteristics.items():
        for i, value in enumerate(values):
            if i < len(sample_ids):  # Only process as many values as we have columns
                clinical_data.loc[row_num, sample_ids[i]] = value
    
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    print("Preview of the clinical data:")
    preview = preview_df(selected_clinical_df)
    print(preview)
    
    # Save the clinical data to the specified output file
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of the clinical data:
{'GSM1': [1.0], 'GSM2': [1.0], 'GSM3': [0.0], 'GSM4': [nan], 'GSM5': [nan], 'GSM6': [nan], 'GSM7': [nan], 'GSM8': [nan], 'GSM9': [nan], 'GSM10': [nan], 'GSM11': [nan], 'GSM12': [nan], 'GSM13': [nan], 'GSM14': [nan], 'GSM15': [nan], 'GSM16': [nan], 'GSM17': [nan], 'GSM18': [nan], 'GSM19': [nan], 'GSM20': [nan], 'GSM21': [nan], 'GSM22': [nan]}
Clinical data saved to ../../output/preprocess/Eczema/clinical_data/GSE32924.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Eczema/GSE32924/GSE32924_series_matrix.txt.gz
Gene data shape: (54675, 33)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the data
# The identifiers start with numerical prefixes like "1007_s_at", "1053_at", etc.
# These follow the format of Affymetrix probe IDs, not standard human gene symbols
# Human gene symbols would typically be alphanumeric like "BRCA1", "TP53", etc.
# Therefore, these probe IDs will need to be mapped to human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in the gene annotation data
# Based on the preview, 'ID' contains Affymetrix probe IDs matching the gene expression data
# 'Gene Symbol' contains the human gene symbols we need to map to
probe_id_col = 'ID'
gene_symbol_col = 'Gene Symbol'

print(f"Using {probe_id_col} for probe IDs and {gene_symbol_col} for gene symbols")

# 2. Get a gene mapping dataframe with the probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("Sample of mapping dataframe:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
print("Converting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 5 gene symbols after mapping:")
print(gene_data.index[:5])

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Using ID for probe IDs and Gene Symbol for gene symbols
Mapping dataframe shape: (45782, 2)
Sample of mapping dataframe:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Converting probe-level measurements to gene expression data...


Gene expression data shape after mapping: (21278, 33)
First 5 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Eczema/gene_data/GSE32924.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Check first if we need to complete the clinical feature extraction from Step 2
print("Checking if clinical data extraction is needed...")
if not os.path.exists(out_clinical_data_file):
    print("Clinical data file not found. Extracting clinical features from original data...")
    # Get the matrix file path
    _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Get the clinical data from the matrix file
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    _, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Define conversion functions from Step 2
    def convert_trait(value: str) -> Optional[int]:
        if value is None:
            return None
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'eczema' in value.lower():
            return 1  # Case
        elif 'control' in value.lower() or 'non-involved' in value.lower():
            return 0  # Control
        else:
            return None  # Other conditions like psoriasis

    def convert_age(value: str) -> Optional[float]:
        if value is None:
            return None
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        age_match = re.search(r'(\d+)', value)
        if age_match:
            return float(age_match.group(1))
        return None

    def convert_gender(value: str) -> Optional[int]:
        if value is None:
            return None
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'female' in value.lower():
            return 0
        elif 'male' in value.lower():
            return 1
        return None
    
    # Extract clinical features with identified rows from Step 2
    trait_row = 1
    age_row = 4
    gender_row = 3
    
    clinical_features = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features extracted and saved to: {out_clinical_data_file}")
else:
    print(f"Clinical data file already exists at: {out_clinical_data_file}")
    clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)

# Now proceed with Step 7 as originally planned
# 1. Normalize gene symbols using NCBI Gene database information
print("\nNormalizing gene symbols...")
try:
    # Load the gene data if needed
    if 'gene_data' not in locals() or gene_data is None:
        gene_data = pd.read_csv(out_gene_data_file, index_col=0)
        
    # Normalize gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"Sample of normalized gene symbols: {normalized_gene_data.index[:10].tolist()}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")

# 2. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
        print(linked_data.iloc[:5, :5])
    else:
        print(linked_data)
    
    # 4. Handle missing values
    print("\nHandling missing values...")
    linked_data_clean = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")
    
    # 5. Check for bias in the dataset
    print("\nChecking for bias in dataset features...")
    is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)
    
    # 6. Conduct final quality validation
    note = "Dataset contains gene expression data from skin biopsies comparing different skin conditions including eczema (atopic dermatitis and contact eczema) against other conditions like psoriasis and healthy controls."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data_clean,
        note=note
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data_clean.to_csv(out_data_file, index=True)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset deemed not usable for associative studies. Linked data not saved.")
        
except Exception as e:
    print(f"Error processing data: {e}")
    # If processing fails, we should still validate the dataset status
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,  # We know trait data is available from step 2
        is_biased=True,  # Set to True to ensure it's not marked usable
        df=pd.DataFrame(),  # Empty dataframe since processing failed
        note=f"Failed to process data: {e}"
    )
    print("Dataset validation completed with error status.")

Checking if clinical data extraction is needed...
Clinical data file already exists at: ../../output/preprocess/Eczema/clinical_data/GSE32924.csv

Normalizing gene symbols...
Gene data shape after normalization: (19845, 33)
Sample of normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Normalized gene data saved to ../../output/preprocess/Eczema/gene_data/GSE32924.csv

Linking clinical and genetic data...
Linked data shape: (55, 19846)
Linked data preview (first 5 rows, 5 columns):
      Eczema  A1BG  A1BG-AS1  A1CF  A2M
GSM1     1.0   NaN       NaN   NaN  NaN
GSM2     1.0   NaN       NaN   NaN  NaN
GSM3     0.0   NaN       NaN   NaN  NaN
GSM4     NaN   NaN       NaN   NaN  NaN
GSM5     NaN   NaN       NaN   NaN  NaN

Handling missing values...
Linked data shape after handling missing values: (0, 1)

Checking for bias in dataset features...
Quartiles for 'Eczema':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Eczema' in this dataset is fine.

Abnormality detected in the cohort: GSE32924. Preprocessing failed.
Dataset deemed not usable for associative studies. Linked data not saved.
